In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import torch
import random
import numpy as np
from lstmppo.init import initialize
from lstmppo.trainer import LSTMPPOTrainer
from lstmppo.types import PolicyEvalInput

In [3]:
sys.argv = [""]
cfg = initialize(seconds_since_epoch=1766683383)

In [ ]:
cfg.num_envs = 1
trainer = LSTMPPOTrainer(cfg)

trainer.policy.eval()
trainer.rollout_phase()

# 2. Get a single recurrent minibatch
#    For num_envs == 1, this should be the whole sequence.
batches = list(trainer.buffer.get_recurrent_minibatches())

assert len(batches) == 1,\
    "For validation, use a single minibatch / single env."

batch = batches[0]

T, B, _ = batch.obs.shape
print(f"T = {T}, B = {B}")

# 3. Re-run the policy over the stored sequence
with torch.no_grad():

    eval_output = \
        trainer.policy.evaluate_actions_sequence(
            PolicyEvalInput(obs=batch.obs,          # (T, B, obs_dim)
                            hxs=batch.hxs,          # (B, H) initial state at t=0
                            cxs=batch.cxs,          # (B, H)
                            actions=batch.actions),  # (T, B, 1)
        )

stored_values = batch.values       # (T, B)
stored_logprobs = batch.logprobs   # (T, B)


tensor([ 0.2491,  0.2496,  0.2499,  0.2436,  0.2313,  0.2268,  0.2055,  0.1985,
         0.1684,  0.1568,  0.1190,  0.1015,  0.0569,  0.0703,  0.0237,  0.0396,
        -0.0076,  0.0026, -0.0460, -0.0419, -0.0922, -0.0946, -0.1196, -0.1234,
        -0.1388, -0.1431, -0.1613, -0.1677, -0.1895, -0.1993, -0.2253, -0.2395,
        -0.2470, -0.2614, -0.2627, -0.2762, -0.2806, -0.2947, -0.3030, -0.3188,
        -0.3313, -0.3499, -0.3668, -0.3679, -0.3858, -0.3765, -0.3941, -0.3876,
        -0.4061, -0.4036, -0.4240, -0.4253, -0.4477, -0.4521, -0.4761, -0.4705,
        -0.4938, -0.4833, -0.5056, -0.4980, -0.5205, -0.5167, -0.5405, -0.5409,
        -0.5518, -0.5535, -0.5586, -0.5607, -0.5676, -0.5709, -0.5806, -0.5860,
        -0.5987, -0.6068, -0.6228, -0.6169, -0.6334, -0.6182, -0.6335, -0.6205,
        -0.6353, -0.6254, -0.6407, -0.6342, -0.6509, -0.6474, -0.6655, -0.6645,
        -0.6838, -0.6855, -0.6908, -0.6930, -0.6919, -0.6939, -0.6951, -0.6973,
        -0.7010, -0.7039, -0.7095, -0.71

In [18]:
es0.obs - es1.obs

tensor([[0., 0.]], device='cuda:0')